# Data Processing
### Goal: Transform raw data into a relational SQLite database with at least two linked tables

##### *Author - Srishti sahu*

s1 - imports

In [7]:
import pandas as pd
import sqlite3
from pathlib import Path

s2 - loading the data

In [9]:
raw_path = Path("../data/raw.csv")
df_raw = pd.read_csv(raw_path)

s3 - cleaning and renaming

In [10]:
df_cleaned = df_raw.drop(columns=["Unnamed: 0", "Unnamed: 7", "Unnamed: 8"])
df_cleaned.columns = ["artist", "painting_name", "colour_code", "brightness", "size_code", "people_near"]

s4 - adding a unique painting id

In [11]:
df_cleaned.insert(0, "painting_id", range(1, len(df_cleaned) + 1))

s5 - lookup table for attributes

In [12]:
attribute_entries = [
    # Size codes
    ("Size", "S", "Small"),
    ("Size", "M", "Medium"),
    ("Size", "L", "Large"),
    # Use of Colour codes
    ("Use of Colour", "L", "Low"),
    ("Use of Colour", "M", "Medium"),
    ("Use of Colour", "H", "High"),
]

df_attributes = pd.DataFrame(attribute_entries, columns=["attribute_type", "code", "description"])

s6 - SQLite DB with explicit schema

In [13]:
db_path = Path("../data/database.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

s7 - creating the table

In [15]:
cursor.execute("DROP TABLE IF EXISTS paintings;")
cursor.execute("DROP TABLE IF EXISTS attributes;")

cursor.execute("""
CREATE TABLE paintings (
    painting_id INTEGER PRIMARY KEY,
    artist TEXT,
    painting_name TEXT,
    colour_code TEXT,
    brightness TEXT,
    size_code TEXT,
    people_near INTEGER
);
""")

s8 - attributes table

In [16]:
cursor.execute("""
CREATE TABLE attributes (
    attribute_type TEXT,
    code TEXT,
    description TEXT,
    PRIMARY KEY (attribute_type, code)
);
""")

s9 - populating

In [17]:
df_cleaned.to_sql("paintings", conn, if_exists="append", index=False)
df_attributes.to_sql("attributes", conn, if_exists="append", index=False)

conn.commit()
conn.close()

s10 - preview 

In [19]:
print("✅ Database with schema created at:", db_path)
display(df_cleaned.head())
display(df_attributes)

✅ Database with schema created at: ../data/database.db


,painting_id,artist,painting_name,colour_code,brightness,size_code,people_near
0,1,Paolo veronese,Scorn,M,Dull,L,1
1,2,Paris Bordone,A pair of lovers,M,Bright,M,1
2,3,Paolo veronese,The family of Darius before Alexander,H,Moderate,L,5
3,4,Jacopo Tintoretto,The origin of Milky Way,H,Bright,L,3
4,5,Jacopo Tintoretto,Saint George and the dragon,H,Moderate,M,2


,attribute_type,code,description
0,Size,S,Small
1,Size,M,Medium
2,Size,L,Large
3,Use of Colour,L,Low
4,Use of Colour,M,Medium
5,Use of Colour,H,High
